In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display
from unidecode import unidecode
import re

In [2]:
def parse_doi(doi:str):
    return re.sub(r'https*\:\/\/[\w\.]+\/','',doi.lower())

def paser_string(string:str):
    data = str(string).lower()
    data = unidecode(data)
    data = re.sub(r'<[^>]+>','',data)
    return data

In [3]:
scopus = list(MongoClient()["scopus_colombia"]["stage"].find({'DOI': {"$ne": np.nan}},
                                                             {"_id":0,'Title':1,'Year':1, 
                                                              'Source title':1,'Authors':1,"DOI":1,
                                                             "Volume":1, "Issue":1, "Page start":1, "Page end":1}))
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}},{"_id":0,"doi":1}))

In [4]:
JSON(openalex[0])

<IPython.core.display.JSON object>

In [5]:
len(openalex)

203863

In [6]:
len(scopus)

143369

In [7]:
%%time
#positives
def get_p(i):    
    for j in openalex:
        if parse_doi(i["DOI"]) ==  parse_doi(j['doi']):
            return i
p = Parallel(n_jobs=20,backend='multiprocessing',verbose=5)(delayed(get_p)(rec) for rec in scopus)


[Parallel(n_jobs=20)]: Using backend MultiprocessingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  33 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done 157 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 288 tasks      | elapsed:   23.3s
[Parallel(n_jobs=20)]: Done 492 tasks      | elapsed:   33.0s
[Parallel(n_jobs=20)]: Done 725 tasks      | elapsed:   40.9s
[Parallel(n_jobs=20)]: Done 962 tasks      | elapsed:   55.2s
[Parallel(n_jobs=20)]: Done 1380 tasks      | elapsed:  1.2min
[Parallel(n_jobs=20)]: Done 1698 tasks      | elapsed:  1.3min
[Parallel(n_jobs=20)]: Done 2040 tasks      | elapsed:  1.5min
[Parallel(n_jobs=20)]: Done 2418 tasks      | elapsed:  1.8min
[Parallel(n_jobs=20)]: Done 2832 tasks      | elapsed:  2.0min
[Parallel(n_jobs=20)]: Done 3282 tasks      | elapsed:  2.2min
[Parallel(n_jobs=20)]: Done 3768 tasks      | elapsed:  2.5min
[Parallel(n_jobs=20)]: Done 4290 tasks      | elapsed:  2.8min
[Parallel(n_jobs=20)]: Done 4848 tas

CPU times: user 14min 55s, sys: 2min 12s, total: 17min 7s
Wall time: 3h 16min 28s


[Parallel(n_jobs=20)]: Done 143369 out of 143369 | elapsed: 196.5min finished


In [8]:
p.count(None)

41596

In [9]:
p=[i for i in p if i is not None ]

In [10]:
p[0]

{'Authors': 'Jaramillo-Ayerbe F., Berrío-Muñoz J.',
 'Title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids',
 'Year': 1998,
 'Source title': 'Archives of Dermatology',
 'Volume': '134',
 'Issue': '2',
 'Page start': '143',
 'Page end': '145',
 'DOI': '10.1001/archderm.134.2.143'}

In [11]:
#los que no tienen un doi en oa, osea los que no están en p_dois

p_dois = [i["DOI"] for i in p]

In [12]:
len(p_dois)

101773

In [13]:
%%time
#negatives
n=[]
counter=0
n_dois = []
for i in scopus:
    if i["DOI"] not in p_dois:
        n.append(i)
        counter+=1
        if counter %10 == 0:
            clear_output(wait=True)
            print(counter)

41590
CPU times: user 6min 27s, sys: 6.43 s, total: 6min 33s
Wall time: 6min 29s


In [14]:
len(p)

101773

In [15]:
len(n)

41596

In [16]:
len(scopus)

143369

In [17]:
len(p)+len(n)

143369

In [18]:
with open('dataset_full.pkl', 'wb') as f:
    pickle.dump({"p":p,"n":n}, f)